# Dewan Lab Image Analysis

## 0: Run once to create all needed directories at beginning of a project

In [ ]:
from Python.Helpers import DewanIOhandler
DewanIOhandler.createProjectFramework()

## 1: Always Execute! Load Libraries and User Settings

### 1a: Import Libraries

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
from tqdm import trange
from Python.Helpers import DewanDataStore
from Python.Helpers import DewanImportDataHelper
from Python.Helpers import DewanIOhandler
from Python import DewanAUROC
from Python import DewanPlotting
from Python import DewanStats
from Python import DewanManualCuration

### 1b: User Configurables

In [3]:
animal = 'VGAT5'
date = '11-16-22'

# Settings for Inscopix box
LED_power = 1
GAIN = 2.2
FOCUS = 250

PreTrialTime = 3.5  # Imaging time before the final valve opens
PostTrialTime = 3.5  # Imaging time after final valve closes

# Criterion for determining good trials vs bad trials, will be used later
goodCriterion = 0.5
# in the findGoodTrials segment


# Configurables for AUROC
baselineDuration = 2  # number of seconds before the fvOnTime
responseDuration = 2  # number of seconds after the fvOnTime
PlotFigures = True

fileHeader = animal + '-' + date + '-'


## 2: Data Import and Processing

### 2a: Import and pre-process the raw data

In [35]:
# Read in data for processing.  Needs Cell Traces, Odor List, and GPIO file.
folder = './ImagingAnalysis/RawData/'

In [ ]:

CellTraceData = pd.read_csv(
    f'{folder}/CellTraces.csv', engine='pyarrow')

In [36]:
OdorData = np.array(pd.read_excel(
    f'{folder}/OdorList.xlsx', usecols=[0], header=None))

In [37]:
OdorData

array([['4AL'],
       ['5AMINE'],
       ['4AMINE'],
       ['7ATE'],
       ['MO'],
       ['25ONE'],
       ['6OL'],
       ['6OL'],
       ['4AMINE'],
       ['7ATE'],
       ['MO'],
       ['5AMINE'],
       ['25ONE'],
       ['4AL'],
       ['4AMINE'],
       ['6OL'],
       ['MO'],
       ['4AL'],
       ['5AMINE'],
       ['7ATE'],
       ['25ONE'],
       ['Buzzer'],
       ['25ONE'],
       ['4AL'],
       ['4AMINE'],
       ['7ATE'],
       ['5AMINE'],
       ['MO'],
       ['6OL'],
       ['4AL'],
       ['7ATE'],
       ['25ONE'],
       ['5AMINE'],
       ['6OL'],
       ['4AMINE'],
       ['MO'],
       ['4AL'],
       ['7ATE'],
       ['MO'],
       ['6OL'],
       ['4AMINE'],
       ['5AMINE'],
       ['25ONE'],
       ['Buzzer'],
       ['MO'],
       ['25ONE'],
       ['7ATE'],
       ['4AMINE'],
       ['5AMINE'],
       ['6OL'],
       ['4AL'],
       ['7ATE'],
       ['5AMINE'],
       ['25ONE'],
       ['MO'],
       ['4AMINE'],
       ['4AL'],
       ['6OL'],
  

In [52]:
GPIOData = pd.read_csv(f'{folder}/GPIO.csv', header=None, engine='pyarrow')
AllCellProps = pd.read_csv(f'{folder}/Props.csv', header=0, engine='pyarrow')

In [53]:
CellTraceData.drop([0], inplace=True)
CellTraceData.set_index(CellTraceData.columns[0], inplace=True)

CellKeys, CellOutlines = DewanIOhandler.get_outline_coordinates()
# Get indicies where there are only one cell part
OnePieceCells = np.where(AllCellProps['NumComponents'] == 1)[0]
CellList = np.arange(len(OnePieceCells))
# Filter out all the bad cell trace data, leaving only the good cells
CellList = CellList[OnePieceCells]
CellKeys = CellKeys[OnePieceCells]
CellTraceData = CellTraceData.apply(pd.to_numeric, errors='coerce')  # Convert everything to numbers
#CellTraceData = np.array(CellTraceData)  # Convert pandas DF to np.array

In [55]:
MaxProjectionImage = DewanManualCuration.generate_max_projection(AllCellProps, CellKeys, CellOutlines, scale_val=1.5)

GoodCells = DewanManualCuration.manual_curation_gui(CellList, CellTraceData, MaxProjectionImage)
if GoodCells is None:
    print('Error!')

In [56]:
GoodCellProperties = AllCellProps.iloc[GoodCells,:]
GoodCellProperties.reset_index()
CellList = CellList[GoodCells]

In [58]:
GoodCellTraceData = CellTraceData.iloc[:, GoodCells]
CellTraceData.iloc[:, 0] = CellTraceData.iloc[:, 0].round(2)  # Round the times to 0.2
GPIOData.iloc[:, 1] = GPIOData.iloc[:, 1].str.replace(' ', '')  # Remove Random Spaces

Index(['1666014314.816', '1666014314.86596', '1666014314.91592',
       '1666014314.96588', '1666014315.015841', '1666014315.0658',
       '1666014315.115761', '1666014315.165721', '1666014315.215681',
       '1666014315.265641',
       ...
       '1666018535.395057', '1666018535.445017', '1666018535.494977',
       '1666018535.544938', '1666018535.594898', '1666018535.644858',
       '1666018535.694818', '1666018535.744778', '1666018535.794738',
       '1666018535.844698'],
      dtype='object', name=' ', length=84489)


In [60]:
GPIO1 = np.array(GPIOData.iloc[:, 1] == "GPIO-1")  # Sniff Sensor Data
GPIO2 = np.array(GPIOData.iloc[:, 1] == "GPIO-2")  # FV Actuation Data
# GPIO3 = np.array(GPIOData.iloc[:,1] == "GPIO-3")  # Running Wheel Data
# GPIO4 = np.array(GPIOData.iloc[:,1] == "GPIO-4")  # Lick Data

In [61]:
#SniffTable = np.array(GPIOData.iloc[GPIO1,:])
FVData = np.array(GPIOData.iloc[GPIO2,:])

In [0]:
# Pickle the reorganized CellTraceData incase its needed later
# Saves Cell Traces, GPIO, Odor List, Sniff, FV data, Good Cell Properties, and the list of Good Cells

# Once these have been saved, they don't need to be re-run on the same data again unless
# the data itself is changed

folder = './ImagingAnalysis/PreProcessedData'

DewanIOhandler.saveDataToDisk(
    CellTraceData, 'CellTraceData', fileHeader, folder)
DewanIOhandler.saveDataToDisk(GPIOData, 'GPIOData', fileHeader, folder)

In [38]:

DewanIOhandler.saveDataToDisk(OdorData, 'OdorData', fileHeader, folder)

VGAT5-11-16-22-OdorData has been saved!


In [62]:

#DewanIOhandler.saveDataToDisk(SniffTable, 'SniffTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(FVData, 'FVdata', fileHeader, folder)
DewanIOhandler.saveDataToDisk(GoodCellProperties, 'GoodCellProperties', fileHeader, folder)
DewanIOhandler.saveDataToDisk(CellList, 'CellList', fileHeader, folder)


VGAT5-11-16-22-CellTraceData has been saved!
VGAT5-11-16-22-GPIOData has been saved!
VGAT5-11-16-22-OdorData has been saved!
VGAT5-11-16-22-FVdata has been saved!
VGAT5-11-16-22-GoodCellProperties has been saved!
VGAT5-11-16-22-CellList has been saved!


### 2b: Checkpoint: Load Preprocessed Data

In [5]:
# Opens the saved pickle files.  If the files have already been saved, code can be re-run
# starting from this point
folder = 'ImagingAnalysis\PreProcessedData'

CellTraceData = DewanIOhandler.loadDataFromDisk(
    'CellTraceData', fileHeader, folder)
FVData = DewanIOhandler.loadDataFromDisk('FVData', fileHeader, folder)
GPIOData = DewanIOhandler.loadDataFromDisk('GPIOData', fileHeader, folder)
OdorData = DewanIOhandler.loadDataFromDisk('OdorData', fileHeader, folder)
# SniffTable = DewanIOhandler.loadDataFromDisk('SniffTable', fileHeader, folder)
GoodCellProperties = DewanIOhandler.loadDataFromDisk(
    'GoodCellProperties', fileHeader, folder)
CellList = DewanIOhandler.loadDataFromDisk('CellList', fileHeader, folder)


VGAT5-11-16-22-CellTraceData has loaded successfully!
VGAT5-11-16-22-FVData has loaded successfully!
VGAT5-11-16-22-GPIOData has loaded successfully!
VGAT5-11-16-22-OdorData has loaded successfully!
VGAT5-11-16-22-GoodCellProperties has loaded successfully!
VGAT5-11-16-22-CellList has loaded successfully!


### 2c: Cell Data Complilation

In [26]:
# Parses the final valve data to identify when the final valve is open vs when it is closed.
FinalValveValues = FVData[:, 2].astype(float)
NumberOfValues = len(FinalValveValues)
ValveStatus = 0
FinalValveOffIndex = []
FinalValveOnIndex = []
for i in trange((NumberOfValues - 1), desc="Processing: "):
    ValveDataDiff = FinalValveValues[i + 1] - FinalValveValues[i]

    if ValveStatus == 0:
        if ValveDataDiff > 10000:
            FinalValveOnIndex.append(i + 1)
            ValveStatus = 1
    else:
        if ValveDataDiff < -10000:
            FinalValveOffIndex.append(i)
            ValveStatus = 0

FinalValveOffIndex = np.array(FinalValveOffIndex)  # verified to match MTLB
FinalValveOnIndex = np.array(FinalValveOnIndex)  # verified to match MTLB

Processing: 100%|██████████| 7997935/7997935 [00:05<00:00, 1561985.21it/s]


In [19]:
FinalValveOffIndex


array([   9998,   32523,   55662,   80706,  103588,  126859,  152393,
        175741,  199796,  223411,  249219,  274577,  300412,  326405,
        350473,  374703,  399032,  425548,  452130,  478443,  502801,
        530222,  556082,  580061,  604016,  630340,  654453,  678755,
        703148,  727303,  751694,  777641,  801962,  825837,  850271,
        876768,  902406,  928441,  954653,  978882, 1003417, 1029597,
       1054335, 1081549, 1105472, 1130776, 1157378, 1183301, 1207904,
       1232100, 1256420, 1281006, 1306977, 1330942, 1356775, 1380413,
       1404182, 1428128, 1453818, 1477582, 1501530, 1527175, 1551462,
       1578395, 1604449, 1632059, 1658792, 1683476, 1709772, 1735973,
       1762517, 1788669, 1812617, 1839365, 1863533, 1888395, 1912788,
       1938914, 1964804, 1991332, 2017397, 2043763, 2068309, 2092440,
       2118358, 2142548, 2972107, 2999722, 3026221, 3050461, 3076871,
       3102617, 3127116, 3151242, 3177751, 3205014, 3231392, 3256305,
       3280634, 3307

In [28]:
# creates a list of data for the start and end time points of trials
# eliminates bad trials (trials with too short/long of a trial period etc.)
# searches through the start and ends of the good trials for matching timepoints
# and outputs indices for the on/off points of good trials
TrialStartTimes = np.array(FVData[FinalValveOnIndex, 0].astype(float) - PreTrialTime)
TrialEndTimes = np.array(FVData[FinalValveOffIndex, 0].astype(float) + PostTrialTime)
CellTraceOnIndexes, GoodTrialsOnTimes = DewanImportDataHelper.findGoodTrials(CellTraceData, TrialStartTimes, goodCriterion)
CellTraceOffIndexes, GoodTrialsOffTime = DewanImportDataHelper.findGoodTrials(CellTraceData, TrialEndTimes, goodCriterion)
GoodTrials, GoodIndexesOn, GoodIndexesOff = DewanImportDataHelper.intersect_matlab(GoodTrialsOnTimes, GoodTrialsOffTime)


In [33]:
print(GoodTrials)

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153.
 154. 155. 156. 157. 158. 159. 160. 161. 162. 163. 164. 165. 166. 167.
 168. 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180. 181.
 182. 183. 184. 185. 186. 187. 188. 189. 190. 191. 192. 193. 194. 195.
 196. 

In [39]:
# Index all data by the good trials to exclude data from bad trials

CellTraceOnIndexes = CellTraceOnIndexes[GoodIndexesOn.astype(int)]
CellTraceOffIndexes = CellTraceOffIndexes[GoodIndexesOff.astype(int)]

# https://knowyourmeme.com/photos/1402268 ?????
# goodIdx = GoodTrials.astype(int) - 1

FinalValveOnIndex = FinalValveOnIndex[GoodTrials.astype(int)]
FinalValveOffIndex = FinalValveOffIndex[GoodTrials.astype(int)]
TrialStartTimes = TrialStartTimes[GoodTrials.astype(int)]
TrialEndTimes = TrialEndTimes[GoodTrials.astype(int)]
OdorData = OdorData[GoodTrials.astype(int)]

#### Sniffing Data Compilation

In [67]:
# # Compiles data for sniffing from good trialss
# SniffingStartIndexes = np.zeros(FinalValveOnIndex.size)
# SniffingEndIndexes = np.zeros(FinalValveOnIndex.size)
#
# for i in trange(len(GoodTrials), desc="Trial: "):
#     TrialStartTime_s = float(FVData[:, 0][FinalValveOnIndex[i]]) - PreTrialTime
#     TrialEndTime_s = float(FVData[:, 0][FinalValveOffIndex[i]]) + PostTrialTime
#     SniffingStartIndexes[i] = len(np.nonzero(
#         SniffTable[:, 0].astype(float) < TrialStartTime_s)[0])
#     SniffingEndIndexes[i] = len(np.nonzero(
#         SniffTable[:, 0].astype(float) < TrialEndTime_s)[0]) - 1
#
# NumSniffPoints = np.max(SniffingEndIndexes - SniffingStartIndexes)
#
# SniffData = np.zeros((NumSniffPoints.astype(int), len(SniffingStartIndexes)))
#
# for j in trange(len(SniffingStartIndexes), desc="Trial: "):
#     SniffLength = SniffingEndIndexes[j] - SniffingStartIndexes[j]
#     SniffData[0:int(SniffLength), j] = SniffTable[SniffingStartIndexes[j].astype(
#         int):SniffingEndIndexes[j].astype(int), 2]
#
# DewanIOhandler.saveDataToDisk(
#     SniffData, 'SniffData', fileHeader, 'PreProcessedData')


In [69]:
print(CellTraceOffIndexes[-2])
print(CellTraceOnIndexes[-2])

IndexError: index -2 is out of bounds for axis 0 with size 0

### 2d: Gather all **cell v. time v. trial** data into single array

In [41]:
MaxDataPoints = np.abs(np.max(CellTraceOffIndexes - CellTraceOnIndexes))

Data = np.zeros((len(CellList), len(CellTraceOffIndexes), MaxDataPoints.astype(int)))

for cell in range(len(CellList)):
    for trial in range(len(CellTraceOnIndexes)):
        DataSize = (CellTraceOffIndexes[trial] - CellTraceOnIndexes[trial])
        Data[cell, trial, :DataSize[0].astype(int)] = CellTraceData.iloc[CellTraceOnIndexes[trial][0].astype(int):CellTraceOffIndexes[trial][0].astype(int), cell + 1]

Data = np.delete(Data, 180, 2)
Baseline_Shift_Data = Data + abs(np.min(Data))


In [59]:
# Map of trial times relative to startTrial time
FinalValveTimeMap = np.zeros((len(CellTraceOffIndexes), MaxDataPoints.astype(int)))
# Map of times per trial in unix time for AUROC
UnixTimeArray = np.zeros((len(CellTraceOffIndexes), max(
    CellTraceOffIndexes - CellTraceOnIndexes).astype(int)[0]))

for trial in range(len(CellTraceOffIndexes)):
    DataSize = (CellTraceOffIndexes[trial] - CellTraceOnIndexes[trial])
    FinalValveData = np.array(CellTraceData.index.values[CellTraceOnIndexes[trial][0].astype(int):CellTraceOffIndexes[trial][0].astype(int)]).astype(float)
    Offset = float(FVData[FinalValveOnIndex[trial], 0])
    FinalValveTimeMap[trial, :DataSize[0].astype(int)] = FinalValveData - Offset
    UnixTimeArray[trial, :DataSize[0].astype(int)] = FinalValveData

FinalValveTimeMap = np.delete(FinalValveTimeMap, 180, 1)
UnixTimeArray = np.delete(UnixTimeArray, 180, 1)


[[-3.48979878 -3.43983889 -3.38987899 -3.33991885 -3.28995895 -3.23999786
  -3.19003797 -3.14007783 -3.09011793 -3.04015779 -2.9901979  -2.940238
  -2.89027786 -2.84031796 -2.79035783 -2.74039793 -2.69043684 -2.6404779
  -2.59051681 -2.54055691 -2.49059701 -2.44063687 -2.39067698 -2.34071684
  -2.29075694 -2.2407968  -2.19083595 -2.14087582 -2.09091592 -2.04095578
  -1.99099588 -1.94103599 -1.89107585 -1.84111595 -1.79115582 -1.74119592
  -1.69123483 -1.64127588 -1.59131479 -1.54135489 -1.491395   -1.44143486
  -1.39147496 -1.34151483 -1.29155493 -1.24159479 -1.19163489 -1.1416738
  -1.09171391 -1.04175401 -0.99179387 -0.94183397 -0.89187384 -0.84191394
  -0.7919538  -0.7419939  -0.69203401 -0.64207292 -0.59211278 -0.54215288
  -0.49219298 -0.44223285 -0.39227295 -0.34231281 -0.29235291 -0.24239278
  -0.19243288 -0.14247298 -0.09251189 -0.04255199  0.00740814  0.05736804
   0.10732818  0.15728807  0.20724821  0.25720811  0.30716801  0.3571291
   0.40708899  0.45704913  0.50700903  0.55

### 2e: Combined Data Output

In [60]:
OdorIndexes = []

for odor in OdorData:
    OdorIndexes.append(odor[0])

columnNames = ['CentroidX', 'CentroidY', 'NumComponents', 'Size']
TableOfContents = pd.DataFrame(GoodCellProperties.iloc[:, 5:9], index = CellList, columns=columnNames)

In [61]:

path = f'./ImagingAnalysis/CombinedData/{fileHeader}CombinedData.xlsx'
writer = pd.ExcelWriter(path, engine='xlsxwriter')

TableOfContents.to_excel(writer, sheet_name='TOC')
TimeMapSheet = pd.DataFrame(
    FinalValveTimeMap[:, :180], np.arange(1, len(CellTraceOffIndexes) + 1))
TimeMapSheet.to_excel(writer, sheet_name='TimeMap')

for i in trange(len(CellList), desc="Writing Cell: "):
    DataDF = pd.DataFrame(Baseline_Shift_Data[i][:][:], OdorIndexes)
    #DataDf.index = indexes
    DataDF.to_excel(writer, sheet_name='Cell %d' % CellList[i])

writer.close()

#headers = []

# for i in range(len(Sniff[0,:])):
# headers.append('Trial ' + str(i))

#SniffDF = pd.DataFrame(Sniff)
#SniffDF.to_excel('./ImportOutputs/' + fileHeader + 'SniffData.xlsx', sheet_name = 'Data', header=headers)


Writing Cell: 100%|██████████| 18/18 [00:11<00:00,  1.58it/s]


### 2f: Save information needed for AUROC

In [62]:
DewanIOhandler.saveDataToDisk(Data, 'CombinedData', fileHeader, './ImagingAnalysis/CombinedData')

folder = './ImagingAnalysis/AUROCImports'
DewanIOhandler.saveDataToDisk(FinalValveOnIndex, 'FVonIdx', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    UnixTimeArray, 'UnixTimeArray', fileHeader, folder)
DewanIOhandler.saveDataToDisk(FinalValveTimeMap, 'FVTimeMap', fileHeader, folder)
DewanIOhandler.saveDataToDisk(OdorData, 'OdorData', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    Baseline_Shift_Data, 'Baseline_Shift_Data', fileHeader, folder)


VGAT5-11-16-22-CombinedData has been saved!
VGAT5-11-16-22-FVonIdx has been saved!
VGAT5-11-16-22-UnixTimeArray has been saved!
VGAT5-11-16-22-FVTimeMap has been saved!
VGAT5-11-16-22-OdorData has been saved!
VGAT5-11-16-22-Baseline_Shift_Data has been saved!


## 3: AUROC

### 3a: Import Data 

In [49]:
Data = DewanIOhandler.loadDataFromDisk('CombinedData', fileHeader, './ImagingAnalysis/CombinedData')

folder = 'ImagingAnalysis/PreProcessedData'
FVData = DewanIOhandler.loadDataFromDisk('FVData', fileHeader, folder)
CellList = DewanIOhandler.loadDataFromDisk('CellList', fileHeader, folder)

folder = 'ImagingAnalysis/AUROCImports'
FinalValveOnIndex = DewanIOhandler.loadDataFromDisk('FVonIdx', fileHeader, folder)
UnixTimeArray = DewanIOhandler.loadDataFromDisk(
    'UnixTimeArray', fileHeader, folder)
OdorData = DewanIOhandler.loadDataFromDisk('OdorData', fileHeader, folder)
FinalValveTimeMap = DewanIOhandler.loadDataFromDisk('FVTimeMap', fileHeader, folder)
Baseline_Shift_Data = DewanIOhandler.loadDataFromDisk(
    'Baseline_Shift_Data', fileHeader, folder)


VGAT5-11-16-22-CombinedData has loaded successfully!
VGAT5-11-16-22-FVData has loaded successfully!
VGAT5-11-16-22-CellList has loaded successfully!
VGAT5-11-16-22-FVonIdx has loaded successfully!
VGAT5-11-16-22-UnixTimeArray has loaded successfully!
VGAT5-11-16-22-OdorData has loaded successfully!
VGAT5-11-16-22-FVTimeMap has loaded successfully!
VGAT5-11-16-22-Baseline_Shift_Data has loaded successfully!


### 3b: Run AUROC

In [0]:
AUROCData = DewanDataStore.AUROCdataStore(
    Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FinalValveOnIndex, UnixTimeArray, baselineDuration, responseDuration, PlotFigures)
onTimeReturnValues = DewanAUROC.AUROC(AUROCData, False)

In [65]:

latentReturnValues = DewanAUROC.AUROC(AUROCData, True)

In [66]:
onTimeAUROCSignificanceTable = []
onTimeAUROCValueTable = []
onTimeAUROC_Ubounds = []
onTimeAUROC_Lbounds = []
onTimeAUROC_prctile = []


latentAUROCSignificanceTable = []
latentAUROCValueTable = []
latentAUROC_Ubounds = []
latentAUROC_Lbounds = []
latentAUROC_prctile = []

baseline_start_indexes = []
baseline_end_indexes = []
ontime_evoked_start_indexes = []
ontime_evoked_end_indexes = []
latent_evoked_start_indexes = []
latent_evoked_end_indexes = []

for returns in onTimeReturnValues:
    onTimeAUROCSignificanceTable.append(returns.response_chart)
    onTimeAUROCValueTable.append(returns.auroc_values)
    onTimeAUROC_Ubounds.append(returns.all_upper_bounds)
    onTimeAUROC_Lbounds.append(returns.all_lower_bounds)
    onTimeAUROC_prctile.append(returns.percentiles)
    baseline_start_indexes.append(returns.baseline_start_indexes)
    baseline_end_indexes.append(returns.baseline_end_indexes)
    ontime_evoked_start_indexes.append(returns.evoked_start_indexes)
    ontime_evoked_end_indexes.append(returns.evoked_end_indexes)


for returns in latentReturnValues:
    latentAUROCSignificanceTable.append(returns.response_chart)
    latentAUROCValueTable.append(returns.auroc_values)
    latentAUROC_Ubounds.append(returns.all_upper_bounds)
    latentAUROC_Lbounds.append(returns.all_lower_bounds)
    latentAUROC_prctile.append(returns.percentiles)
    latent_evoked_start_indexes.append(returns.evoked_start_indexes)
    latent_evoked_end_indexes.append(returns.evoked_end_indexes)

onTimeAUROCSignificanceTable = np.array(onTimeAUROCSignificanceTable)
latentAUROCSignificanceTable = np.array(latentAUROCSignificanceTable)
onTimeAUROCSignificanceTable = onTimeAUROCSignificanceTable.astype(float)
latentAUROCSignificanceTable = latentAUROCSignificanceTable.astype(float)

#baseline_start_indexes = np.array(baseline_start_indexes)
# baseline_end_indexes = np.array(baseline_end_indexes)
# ontime_evoked_start_indexes = np.array(ontime_evoked_start_indexes)
# ontime_evoked_end_indexes = np.array(ontime_evoked_end_indexes)
# latent_evoked_start_indexes = np.array(latent_evoked_start_indexes)
# latent_evoked_end_indexes = np.array(latent_evoked_end_indexes)

for rows in range(len(latentAUROCSignificanceTable)):
    evokedCells = np.nonzero(onTimeAUROCSignificanceTable[rows] != 0)
    latentAUROCSignificanceTable[rows][evokedCells] = 0

### 3c: Save AROC

In [67]:
folder = 'ImagingAnalysis/AUROCData'

DewanIOhandler.saveDataToDisk(
    onTimeAUROCSignificanceTable, 'onTimeAUROCSignificanceTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROCValueTable, 'onTimeAUROCValueTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROC_Lbounds, 'onTimeAUROC_Lbounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROC_Ubounds, 'onTimeAUROC_Ubounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    onTimeAUROC_prctile, 'onTimeAUROC_prctile', fileHeader, folder)

DewanIOhandler.saveDataToDisk(
    baseline_start_indexes, 'baseline_start_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    baseline_end_indexes, 'baseline_end_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    ontime_evoked_start_indexes, 'ontime_evoked_start_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    ontime_evoked_end_indexes, 'ontime_evoked_end_indexes', fileHeader, folder)

DewanIOhandler.saveDataToDisk(
    latentAUROCSignificanceTable, 'latentAUROCSignificanceTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROCValueTable, 'latentAUROCValueTable', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROC_Lbounds, 'latentAUROC_Lbounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROC_Ubounds, 'latentAUROC_Ubounds', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latentAUROC_prctile, 'latentAUROC_prctile', fileHeader, folder)

DewanIOhandler.saveDataToDisk(
    latent_evoked_start_indexes, 'latent_evoked_start_indexes', fileHeader, folder)
DewanIOhandler.saveDataToDisk(
    latent_evoked_end_indexes, 'latent_evoked_end_indexes', fileHeader, folder)

VGAT5-11-16-22-onTimeAUROCSignificanceTable has been saved!
VGAT5-11-16-22-onTimeAUROCValueTable has been saved!
VGAT5-11-16-22-onTimeAUROC_Lbounds has been saved!
VGAT5-11-16-22-onTimeAUROC_Ubounds has been saved!
VGAT5-11-16-22-onTimeAUROC_prctile has been saved!
VGAT5-11-16-22-baseline_start_indexes has been saved!
VGAT5-11-16-22-baseline_end_indexes has been saved!
VGAT5-11-16-22-ontime_evoked_start_indexes has been saved!
VGAT5-11-16-22-ontime_evoked_end_indexes has been saved!
VGAT5-11-16-22-latentAUROCSignificanceTable has been saved!
VGAT5-11-16-22-latentAUROCValueTable has been saved!
VGAT5-11-16-22-latentAUROC_Lbounds has been saved!
VGAT5-11-16-22-latentAUROC_Ubounds has been saved!
VGAT5-11-16-22-latentAUROC_prctile has been saved!
VGAT5-11-16-22-latent_evoked_start_indexes has been saved!
VGAT5-11-16-22-latent_evoked_end_indexes has been saved!


## 4: Plotting

### 4a: Import Data

In [ ]:
folder = 'ImagingAnalysis/AUROCData'

onTimeAUROCSignificanceTable = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROCSignificanceTable', fileHeader, folder)
onTimeAUROCValueTable = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROCValueTable', fileHeader, folder)
onTimeAUROC_Lbounds = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROC_Lbounds', fileHeader, folder)
onTimeAUROC_Ubounds = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROC_Ubounds', fileHeader, folder)
onTimeAUROC_prctile = DewanIOhandler.loadDataFromDisk(
    'onTimeAUROC_prctile', fileHeader, folder)

latentAUROCSignificanceTable = DewanIOhandler.loadDataFromDisk(
    'latentAUROCSignificanceTable', fileHeader, folder)
latentAUROCValueTable = DewanIOhandler.loadDataFromDisk(
    'latentAUROCValueTable', fileHeader, folder)
latentAUROC_Lbounds = DewanIOhandler.loadDataFromDisk(
    'latentAUROC_Lbounds', fileHeader, folder)
latentAUROC_Ubounds = DewanIOhandler.loadDataFromDisk(
    'latentAUROC_Ubounds', fileHeader, folder)
latentAUROC_prctile = DewanIOhandler.loadDataFromDisk(
    'latentAUROC_prctile', fileHeader, folder)


baseline_start_indexes = DewanIOhandler.loadDataFromDisk(
    'baseline_start_indexes', fileHeader, folder)
baseline_end_indexes = DewanIOhandler.loadDataFromDisk(
    'baseline_end_indexes', fileHeader, folder)
ontime_evoked_start_indexes = DewanIOhandler.loadDataFromDisk(
    'ontime_evoked_start_indexes', fileHeader, folder)
ontime_evoked_end_indexes = DewanIOhandler.loadDataFromDisk(
    'ontime_evoked_end_indexes', fileHeader, folder)
latent_evoked_start_indexes = DewanIOhandler.loadDataFromDisk(
    'latent_evoked_start_indexes', fileHeader, folder)
latent_evoked_end_indexes = DewanIOhandler.loadDataFromDisk(
    'latent_evoked_end_indexes', fileHeader, folder)

Data = DewanIOhandler.loadDataFromDisk(
    'CombinedData', fileHeader, 'ImagingAnalysis/CombinedData')

folder = 'ImagingAnalysis/PreProcessedData'
FVData = DewanIOhandler.loadDataFromDisk('FVData', fileHeader, folder)
CellList = DewanIOhandler.loadDataFromDisk('CellList', fileHeader, folder)

folder = 'ImagingAnalysis/AUROCImports'
FinalValveOnIndex = DewanIOhandler.loadDataFromDisk('FVonIdx', fileHeader, folder)
UnixTimeArray = DewanIOhandler.loadDataFromDisk(
    'UnixTimeArray', fileHeader, folder)
OdorData = DewanIOhandler.loadDataFromDisk('OdorData', fileHeader, folder)
FinalValveTimeMap = DewanIOhandler.loadDataFromDisk('FVTimeMap', fileHeader, folder)
Baseline_Shift_Data = DewanIOhandler.loadDataFromDisk(
    'Baseline_Shift_Data', fileHeader, folder)


### 4b: Cells v. Odors AUROC Significance

In [68]:
onTimePlottingData = DewanDataStore.PlottingDataStore(Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FinalValveTimeMap, FinalValveOnIndex, UnixTimeArray, baselineDuration, responseDuration, onTimeAUROCSignificanceTable, onTimeAUROC_Lbounds, onTimeAUROC_Ubounds, onTimeAUROC_prctile, baseline_start_indexes, baseline_end_indexes, ontime_evoked_start_indexes, ontime_evoked_end_indexes)

latentPlottingData = DewanDataStore.PlottingDataStore(Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FinalValveTimeMap, FinalValveOnIndex, UnixTimeArray, baselineDuration, responseDuration, latentAUROCSignificanceTable, latentAUROC_Lbounds, latentAUROC_Ubounds, latentAUROC_prctile, baseline_start_indexes, baseline_end_indexes, latent_evoked_start_indexes, latent_evoked_end_indexes)

#### 4b-1: Plot Cell v Odor Significance Matricies

In [69]:
DewanPlotting.plotCellvOdorMatricies(onTimePlottingData, False)
DewanPlotting.plotCellvOdorMatricies(latentPlottingData, True)

#### 4b-2: Plot All Cell Traces

In [70]:
# Plot all cells v odors; only needed for AUROC verification
DewanPlotting.plotAllCells(onTimePlottingData, False)
DewanPlotting.plotAllCells(latentPlottingData, True)

#### 4b-3: Plot Significant Cell Traces

In [ ]:
# Oh no! Nothing here yet :(
# https://media.tenor.com/vMvR0gEolCEAAAAC/nothing-to-see-here-explosion.gif

### 4c: Plot Trace Variance

In [ ]:
trialScatterData = DewanDataStore.AUROCdataStore(
    Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FinalValveOnIndex, UnixTimeArray, baselineDuration, responseDuration, False)


DewanPlotting.plotTrialsPerPairing(
    trialScatterData, onTimeAUROCSignificanceTable, False)
DewanPlotting.plotTrialsPerPairing(
    trialScatterData, onTimeAUROCSignificanceTable, True)


## 5. Statistics

### 5a. Load Data

In [ ]:
#Oh no, nothing here yet :(

### 5b. Correlation Coefficients

In [73]:
similarityData = DewanDataStore.AUROCdataStore(Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FinalValveOnIndex, UnixTimeArray, baselineDuration, responseDuration, False)
CCs = DewanStats.crossTrialConsistency(similarityData, onTimeAUROCSignificanceTable, False)

### 5c. Lifetime and Population Sparseness

In [75]:
sparsenessData = DewanDataStore.AUROCdataStore(
    Baseline_Shift_Data, CellList, OdorData, FVData, fileHeader, FinalValveOnIndex, UnixTimeArray, baselineDuration, responseDuration, False)

lifetimeSparseness, cells = DewanStats.lifetimeSparseness(sparsenessData, onTimeAUROCSignificanceTable)
populationSparseness, odors = DewanStats.popSparseness(sparsenessData, onTimeAUROCSignificanceTable)
lsPairs = [f'Cell {cells[i]}: {round(lifetimeSparseness[i],5)}' for i in range(len(cells))]
psPairs = [f'Odor: {sparsenessData.unique_odors[odors[i]]}: {round(populationSparseness[i],5)}' for i in range(len(odors))]

In [76]:

psPairs


['Odor: 25ONE: 1.02714',
 'Odor: 4AL: 1.01189',
 'Odor: 4AMINE: 0.87736',
 'Odor: 4ATE: 1.05234',
 'Odor: 4OL: 1.01256',
 'Odor: 4ONE: 0.9621',
 'Odor: 5AL: 1.05882',
 'Odor: 5AMINE: 0.85745',
 'Odor: 5ATE: 1.05853',
 'Odor: 5OL: 1.02715',
 'Odor: 6AL: 1.02532',
 'Odor: 6AMINE: 0.95931',
 'Odor: 6ATE: 0.87504',
 'Odor: 6OL: 0.93906',
 'Odor: 6ONE: 0.99836',
 'Odor: 7AL: 1.02357',
 'Odor: 7AMINE: 0.9391',
 'Odor: 7ATE: 0.93227',
 'Odor: 7OL: 0.96999',
 'Odor: 7ONE: 1.03056',
 'Odor: Buzzer: 1.03758']